In [146]:
import pydocparser
import requests
from bs4 import BeautifulSoup
import webbrowser
import re

In [ ]:

api_key = 'a97eb5d696857fea8bfa9e57fb9994017a3bb7da'
parser_id = 'paper_parser'

parser = pydocparser.Parser()
parser.login(api_key)
result = parser.ping()
print(result)

In [8]:
id = parser.upload_file_by_path("data_samples/electricity-theft-detection-using-machine-learning-IJERTCONV10IS04024.pdf", "paper_parser") 

In [9]:
data = parser.get_one_result("paper_parser", id)
data

In [11]:
keywords = data[0]['keywords']
references = data[0]['references']
abstract = data[0]['abstract']

"-Energy Meter, Convolutional Neural Network ,                   additionally affect the decisions made through the SO\n   Advanced Meter Infrastructure, System Operators, System                  misleading behaviour does not simplest cause economic\n   Grid                                                                     regarding grid management, which can also reason the\n                     I. INTRODUCTION                                        instability of the grid or blackout in intense cases . energy\n Energy meter, or kilowatt-hour meter is a tool that measures               theft is a critical trouble inside the existing power grid that\nthe amount of electric energy consumed via a house, a                       causes hefty monetary losses. We deal with in this paper is a\ncommercial enterprise,or an electrically powered tool                       way to permit the to be able to display load, compute\n.Electric utilities use power meters installed at purchaser          

In [151]:
def convert_to_file(text, file_path):
    
    with open(file_path, 'w') as file:
        file.write(text)

    return file_path

In [15]:
convert_to_file(references, 'extracted_text/')

'extracted_text/references.txt'

In [158]:
def extract_words_in_quotes(input_string):
    clean = re.sub(r'\n', ' ', input_string)
    pattern = r'[“"]([^”"]*)[”"]'
    matches = re.findall(pattern, clean)
    
    return matches

In [161]:
names = extract_words_in_quotes(references)

Scraping

In [139]:
def get_abstract_from_paper_url(paper_page_url, headers):
    response = requests.get(paper_page_url, headers=headers)

    if response.status_code == 200:
        paper_soup = BeautifulSoup(response.text, 'html.parser')

        abstract_element = paper_soup.find('div', {'class': 'nova-legacy-e-text nova-legacy-e-text--size-m nova-legacy-e-text--family-sans-serif nova-legacy-e-text--spacing-none nova-legacy-e-text--color-grey-800 research-detail-middle-section__abstract'})
        
        if abstract_element:
            abstract = abstract_element.get_text(strip=True)
            return abstract
        else:
            return "Abstract not found."
    else:
        return f"Failed to retrieve paper page: {response.status_code}"

In [145]:
def open_paper_link_by_name(paper_name):
    base_url = "https://www.researchgate.net/search/publication?q="
    search_url = base_url + paper_name.replace(" ", "+")
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    }


    # Make a request to the ResearchGate search page
    response = requests.get(search_url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract all search results
        result_links = soup.find_all('a', {'class':  ['nova-legacy-e-link', 'nova-legacy-e-link--color-inherit', 'nova-legacy-e-link--theme-bare']})

        if result_links:
            print("found")

            for link in result_links:
                result_name = link.get_text(strip=True)
                print(result_name)

                # Check if the result name matches the given paper name
                if paper_name.lower() in result_name.lower():
                    paper_page_url = "https://www.researchgate.net/" + link['href']
                    abstract = get_abstract_from_paper_url(paper_page_url, headers=headers)
                    print("full link: ",paper_page_url)
                    print("abstact: ",abstract)

                    return f"Match found! Opening link for '{result_name}'"
            
            return f"No exact match found for '{paper_name}' in the search results."
        else:
            return f"No results found for '{paper_name}'."
    else:
        return f"Failed to perform the search: {response.status_code}"

# Example usage
paper_name_to_search = "Energy theft detection with energy privacy preservation in the smart grid"
result_message = open_paper_link_by_name(paper_name_to_search)
print(result_message)


found
Energy Theft Detection With Energy Privacy Preservation in the Smart Grid
full link:  https://www.researchgate.net/publication/331578806_Energy_Theft_Detection_With_Energy_Privacy_Preservation_in_the_Smart_Grid?_sg=bR1ptsK9e675dEJxIeZgQmj71xnwJubYsbT9uxlnJoBYreJNZ_vDUmwApR399DD06Hw0wuyhsV09W34
abstact:  As a prominent early instance of the IoT in the smart grid, the advanced metering infrastructure (AMI) provides real-time information from smart meters to both grid operators and customers, exploiting the full potential of demand response. However, the newly-collected information without security protection can be maliciously altered and result in huge loss. In this paper, we propose an energy theft detection scheme with energy privacy preservation in the smart grid. Specially, we use combined convolutional neural networks (CNN) to detect abnormal behavior of the metering data from a long-period pattern observation. In addition, we employ Paillier algorithm to protect the energy p